# Imports

In [6]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from pathlib import Path

In [7]:


# Dossier contenant tes CSV
data_folder = Path("Data")

# Boucle sur chaque fichier CSV du dossier
for csv_path in data_folder.glob("*.csv"):
    # On récupère le nom du fichier sans extension
    var_name = f"{csv_path.stem}_df"
    # On lit le CSV dans un DataFrame
    df = pd.read_csv(csv_path)
    # On crée dynamiquement la variable dans l'espace global
    globals()[var_name] = df

In [8]:
print(globals().keys())  # Affiche les noms des variables créées

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'os', 'dataframes', 'data_folder', 'filename', 'filepath', 'df_name', '_i2', '_i3', 'np', '_i4', 'plt', 'pd', '_i5', '_i6', 'Path', '_i7', 'csv_path', 'var_name', 'df', 'EdStatsCountry_df', 'EdStatsCountry-Series_df', 'EdStatsData_df', 'EdStatsFootNote_df', 'EdStatsSeries_df', '_i8'])
